<a href="https://colab.research.google.com/github/cipB14/Questify/blob/mod3/Blooms_lvl_identifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install transformers

import torch
from google.colab import files
from transformers import  BertTokenizer, BertForSequenceClassification

# 🔹 Load Bloom’s Taxonomy BERT Model
blooms_model_name = "cip29/blooms_bert"  # Update with your model name if different
blooms_tokenizer = BertTokenizer.from_pretrained(blooms_model_name)
blooms_model = BertForSequenceClassification.from_pretrained(blooms_model_name, num_labels=6).to("cuda")



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [9]:
import csv
from google.colab import files
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# 🔹 Function to Classify Questions into Bloom’s Taxonomy Levels
def classify_blooms_taxonomy(question):
    model_name = "cip29/blooms_bert"  # Change to your fine-tuned model
    blooms_tokenizer = BertTokenizer.from_pretrained(model_name)
    blooms_model = BertForSequenceClassification.from_pretrained(model_name, num_labels=6, device_map="cuda")

    inputs = blooms_tokenizer(question, return_tensors="pt", padding=True, truncation=True).to("cuda")
    with torch.no_grad():
        outputs = blooms_model(**inputs)

    probs = torch.nn.functional.softmax(outputs.logits, dim=1).cpu().numpy().flatten()
    bloom_levels = ["BT1 (Remembering)", "BT2 (Understanding)", "BT3 (Applying)",
                    "BT4 (Analyzing)", "BT5 (Evaluating)", "BT6 (Creating)"]

    max_index = probs.argmax()
    predicted_level = bloom_levels[max_index]

    probabilities_dict = {bloom_levels[i]: probs[i] for i in range(len(bloom_levels))}

    return predicted_level, probabilities_dict

# 🔹 Function to Upload & Process CSV
def upload_and_classify_questions():
    uploaded = files.upload()  # Prompt user to upload file
    input_csv = list(uploaded.keys())[0]  # Get the uploaded filename

    output_csv = "classified_questions.csv"

    with open(input_csv, mode='r', encoding='utf-8') as infile, open(output_csv, mode='w', newline='', encoding='utf-8') as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile)

        # Write header row
        writer.writerow(["Question", "Bloom's Taxonomy Level",
                         "BT1 (Remembering)", "BT2 (Understanding)", "BT3 (Applying)",
                         "BT4 (Analyzing)", "BT5 (Evaluating)", "BT6 (Creating)", "Difficulty_Score"])

        next(reader)  # Skip header

        for row in reader:
            question = row[0]  # Extract question
            difficulty_score = row[2] if len(row) > 2 else "N/A"  # Handle missing difficulty score

            bloom_level, probs = classify_blooms_taxonomy(question)

            # Write data to output CSV
            writer.writerow([question, bloom_level,
                             probs["BT1 (Remembering)"], probs["BT2 (Understanding)"],
                             probs["BT3 (Applying)"], probs["BT4 (Analyzing)"],
                             probs["BT5 (Evaluating)"], probs["BT6 (Creating)"], difficulty_score])

    # 📥 Download the classified CSV file
    files.download(output_csv)
    print(f"✅ Classification complete! File saved as {output_csv}")

# 🔹 Run the Function to Upload & Process Questions
upload_and_classify_questions()


Saving dl_questions.csv to dl_questions.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Classification complete! File saved as classified_questions.csv
